## 0) 前提

* エンジン: **MySQL 8.0.40**
* 結合キー:

  * `Sales.product_id` ↔ `Product.product_id`
* 要求出力:

  * 各 `sale_id` ごとに
    `product_name, year, price`
* 集計・順位付けは不要なので **ウィンドウ関数は使わない方がシンプルで高速**

---

## 1) 問題整理

* 入力テーブル

  **Sales**

  * `sale_id` (PK の一部: `sale_id, year`)
  * `product_id` (FK → `Product.product_id`)
  * `year`
  * `quantity`
  * `price`（単価）

  **Product**

  * `product_id` (PK)
  * `product_name`

* 出力仕様

  * 列: `product_name, year, price`
  * 各行は `Sales` の 1 レコードに対応
  * 並び順は任意（`ORDER BY` なし）

* 方針

  * 各売上行に対して、対応する商品の名前を `INNER JOIN` で取得し、そのまま投影

---

## 2) 最適解（単一クエリ）

> 今回は「各行そのまま＋名前付け」なので、**単純な内部結合のみ**が最短・最速です。
> ウィンドウ関数や CTE による前処理は不要です。

```sql
SELECT
  p.product_name,
  s.year,
  s.price
FROM Sales AS s
JOIN Product AS p
  ON p.product_id = s.product_id;

Runtime 1155 ms
Beats 69.04%

```

* `JOIN`（= `INNER JOIN`）なので、`Product` に存在しない `product_id` は自然に除外
* 並び順要件なし → `ORDER BY` を付与しないことで余計なソートコストを回避

---

## 3) 代替解

### 3-1. 相関サブクエリ版（JOIN 不使用）

JOIN が苦手な読者向けの書き方としては、以下のように **相関サブクエリ** も可能です（ただしパフォーマンス面では JOIN が優位）。

```sql
SELECT
  (
    SELECT p.product_name
    FROM Product AS p
    WHERE p.product_id = s.product_id
  ) AS product_name,
  s.year,
  s.price
FROM Sales AS s;

Runtime 5589 ms
Beats 5.01%

```

* `Sales` 側の行ごとに `Product` テーブルを参照
* インデックスがあれば許容されるが、大量データでは JOIN の方が一般的に高速

---

## 4) 要点解説

1. **ID ベースでの結合**

   * 問題文が「`product_id` は `Product` の外部キー」と明示しているため、

     * **結合条件**: `Sales.product_id = Product.product_id`
     * 結果の正しさは ID ベースで保証される

2. **集計・フィルタが不要**

   * 「各 sale_id について」情報を出すだけなので

     * 集約 (`GROUP BY`)
     * 順位付け (`ROW_NUMBER`, `RANK` など)
     * TOP k 抽出 (`LIMIT`)
   は一切不要 → **シンプルな SELECT + JOIN が最適**

3. **`INNER JOIN` を選ぶ理由**

   * 問題文は「Sales テーブルの sale_id ごと」としか書いていないが、同時に `product_id` は `Product` の FK とあるので「整合データ前提」と解釈し、**商品マスタに存在しない ID をわざわざ残す必要はない** ⇒ `INNER JOIN` で十分

4. **並び順任意 → `ORDER BY` 省略**

   * LeetCode 的には「any order」なら `ORDER BY` は不要
   * 省略によりソートステップがなくなり、クエリプランが軽くなる

5. **インデックス利用**

   * 典型的には以下が用意されている想定

     * `Product(product_id)`（PK）
     * `Sales(product_id)`（FK 用インデックス）
   * これにより、`Sales` から `Product` への結合は **ほぼ O(N)** でスキャン可能

---

## 5) 計算量（概算）

* `Sales` の件数を `N`、`Product` の件数を `M` とする
* インデックス付きネストループ / ハッシュ結合を想定

1. **JOIN 処理**

   * `Sales` を 1 度スキャン: O(N)
   * 各 `product_id` に対し `Product` をインデックスルックアップ: O(1) 近似 × N
   * 合計: **O(N)** 近似（`M` がメモリに乗る前提）

2. **追加のソートや集計なし**

   * `GROUP BY` / `ORDER BY` がないため、
     ウィンドウ処理やソート起因の **O(N log N)** コストは発生しない

---

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Sales テーブル]
  B[入力 Product テーブル]
  C[結合 product_id で内部結合]
  D[列選択 product_name year price]
  E[出力 結果テーブル]

  A --> C
  B --> C
  C --> D
  D --> E
```

この問題は「JOIN の基本」を確認するタイプなので、
**余計なウィンドウ関数や CTE を足さず、最短の INNER JOIN で書けるか**がポイントになります。

結論から言うと、この問題に関しては **ほぼ最適** なので、実務目線でも LeetCode 目線でも「これ以上いじる価値がある改善」はありません 👍

---

## 1. 現在のクエリの評価

```sql
SELECT
  p.product_name,
  s.year,
  s.price
FROM Sales AS s
JOIN Product AS p
  ON p.product_id = s.product_id;
```

* やりたいこと:

  * `Sales` の各行に対して、`Product` から `product_name` を引いてくるだけ
* それに対して:

  * 不要な列は出していない
  * 不要な `DISTINCT` / `GROUP BY` / `ORDER BY` もなし
  * 結合条件も PK–FK で素直

**アルゴリズム的には O(N)**（`Sales` を 1 回なめて `Product` にインデックスルックアップ）なので、これ以上いい形はほぼありません。

LeetCode の **Runtime 1155 ms / Beats 69.04%** も、この種の SQL 問題としては十分優秀な部類です。

---

## 2. 相関サブクエリ版が遅い理由

```sql
SELECT
  (
    SELECT p.product_name
    FROM Product AS p
    WHERE p.product_id = s.product_id
  ) AS product_name,
  s.year,
  s.price
FROM Sales AS s;
```

* これは「`Sales` の行ごとに `Product` を探しに行く」構造なので、

  * プランによっては **行数 × サブクエリ** になりやすく、
  * JOIN よりもオーバーヘッドが大きい
* その結果:

  * Runtime 5589 ms / Beats 5.01% と、数字が正直に「非効率です」と言ってます。

**教科書的にも・実務的にも JOIN が正解**なので、ここは今の理解でバッチリです。

---

## 3. 「それでも何かできる？」という観点でのコメント

LeetCode 環境だと、ここから先は **ほぼ誤差の世界** です。

やれるとしてもせいぜい：

* `JOIN` → `INNER JOIN` と明示してもプランはほぼ変わらないはず
* `USING (product_id)` を使って少しだけスッキリ書く程度（性能は同じ）

例:

```sql
SELECT
  p.product_name,
  s.year,
  s.price
FROM Sales AS s
JOIN Product AS p USING (product_id);

Runtime 1306 ms
Beats 31.05%

```

ですが、これは **可読性の好みの問題** であって、実行計画レベルでの改善はほとんど期待できません。

---

## 4. Runtime の数字をどう捉えるか

LeetCode の Runtime は、以下のような要因でブレます：

* 実行サーバの負荷状況
* キャッシュの有無
* 同じクエリでも再実行で数百 ms レベルで揺れることがある

この問題のように

* シンプルな JOIN
* 行数もそこまで多くない
* インデックス前提の PK–FK 結合

というケースでは、**「Beats 69%」なら十分・これ以上追うのは趣味の最適化**という感覚で大丈夫です。

---

## まとめ

* 今の JOIN クエリは **ロジックも計算量もベストプラクティスど真ん中**。
* 相関サブクエリ版が遅いのも理屈どおりで、理解は完璧。
* LeetCode Runtime はノイズも多いので、これ以上は「数字のための微調整」になりがちです。

この問題については、**「もう終わりでよいレベル」**と判断して次の問題に進んでOKだと思います。

